# Solid state computational chemistry

Introductory text.
It is possible to model finite elements. Size of the system is very limited, especially in DFT. In ideal case, the computational time scales 

In [3]:
#IMPORT MODULES
from ase import Atoms
from ase.constraints import FixAtoms
from ase.io import write, read
from ase.optimize import QuasiNewton
from ase.units import Bohr
from ase.visualize import view
from gpaw import GPAW, FermiDirac, PoissonSolver, Mixer, setup_paths, restart
from gpaw.xc.functional import XCFunctional
#from gpaw.xc.hybridg import HybridXC
from gpaw.xc.sic import SIC
from gpaw.test import equal
from gpaw.xc.tools import vxc
from gpaw.wavefunctions.pw import PW
from math import sqrt

In [4]:
#MOLECULE VIEWER CODE
from IPython.display import HTML
def atoms_to_html(atoms):
    'Return the html representation the atoms object as string'

    from tempfile import NamedTemporaryFile

    with NamedTemporaryFile('r+', suffix='.html') as ntf:
        atoms.write(ntf.name, format='html')
        ntf.seek(0)
        html = ntf.read()
    return html

In [5]:
#Define NaCl geometry
pair = Atoms('NaCl', positions=[(0, 0, 0),(0, 0, 2.82)])
pair.set_pbc((False, False, False))
pair.center(vacuum=5)

In [9]:
HTML(atoms_to_html(pair))

In [11]:
#Define calculator and run simple PBE calculation
calc = GPAW(xc='PBE',
            mode='fd',
            h=0.14,
            txt='NaCl-PBE.txt',
            basis='dzp')
pair.set_calculator(calc)
pair.get_potential_energy()
pair.calc.write('NaCl-PBE.gpw', mode='all')
density = calc.get_all_electron_density(gridrefinement=4) * Bohr**3
write('NaCl-PBE.cube', pair, data=density)

/usr/lib/python3/dist-packages/gpaw/fd_operators.py:184: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  a_d, residual, rank, s = np.linalg.lstsq(A_md, [1, 1, 1, 0, 0, 0])


In [21]:
%%bash
. ~/.bashrc
/home/user/LOKT.02.048/week4/bader -p all_atom -p atom_index NaCl-PBE.cube


  GRID BASED BADER ANALYSIS  (Version 0.28a 07/12/12)

  OPEN ... NaCl-PBE.cube       
  GAUSSIAN-STYLE INPUT FILE
  FFT-grid:   288 x 288 x 368
  CLOSE ... NaCl-PBE.cube       

  RUN TIME:   10.36 SECONDS

  CALCULATING BADER CHARGE DISTRIBUTION
                 0  10  25  50  75  100
  PERCENT DONE:  ********************** 

  REFINING AUTOMATICALLY
  ITERATION: 1
  EDGE POINTS:        423840
  REASSIGNED POINTS:   22527
  ITERATION: 2
  CHECKED POINTS:     230618
  REASSIGNED POINTS:       0

  RUN TIME:      75.24 SECONDS

  CALCULATING MINIMUM DISTANCES TO ATOMS
                 0  10  25  50  75  100
  PERCENT DONE:  **********************

  RUN TIME:   10.80 SECONDS

  WRITING BADER ATOMIC CHARGES TO ACF.dat
  WRITING BADER VOLUME CHARGES TO BCF.dat

  NUMBER OF BADER MAXIMA FOUND:              2
      SIGNIFICANT MAXIMA FOUND:              2
           NUMBER OF ELECTRONS:       27.99945

  WRITING ATOMIC VOLUMES 
                 0  10  25  50  75  100
  PERCENT DONE:  ****

In [24]:
f=open("ACF.dat").read()
print(f)

    #         X           Y           Z        CHARGE     MIN DIST    ATOMIC VOL
 --------------------------------------------------------------------------------
    1      9.4486      9.4486      9.4486     10.1083      2.0720   3184.1363
    2      9.4486      9.4486     14.7777     17.8912      3.1912   5467.0589
 --------------------------------------------------------------------------------
    VACUUM CHARGE:               0.0000
    VACUUM VOLUME:               0.0000
    NUMBER OF ELECTRONS:        27.9995



In [6]:
#Define calculator and run PBE calculation with Self-Interaction Error Correction (SIC)
calc = GPAW(#xc=SIC(),
            xc='PBE-PZ-SIC',
            eigensolver='rmm-diis',
            mode='fd',
            h=0.14,
            txt='NaCl-SIC.txt',
            #setups='hgh',
            basis='dzp')
pair.set_calculator(calc)
pair.get_potential_energy()
pair.calc.write('NaCl-SIC.gpw', mode='all')
density = calc.get_all_electron_density(gridrefinement=4) * Bohr**3
write('NaCl-SIC.cube', pair, data=density)

/usr/lib/python3/dist-packages/gpaw/fd_operators.py:184: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  a_d, residual, rank, s = np.linalg.lstsq(A_md, [1, 1, 1, 0, 0, 0])


           UO-0:  0  -4.7               9.49937  CV-S
           UO-0:  1  -4.1               9.45963  CV-S CG
           UO-0:  2  -4.1               9.45737  CV-S CG
           UO-0:  0  -5.5               9.30428  CV-S
           UO-0:  1  -6.9               9.30217  CV-S CG


/usr/lib/python3/dist-packages/gpaw/xc/sic.py:1222: RuntimeWarning: divide by zero encountered in double_scalars
  G       = (E1-E0)/step


           UO-0:  0  -6.6               9.17661  CC-EPS-F
           UO-0:  0  -6.6               9.18834  CC-EPS-F
           UO-0:  0  -6.9               9.25567  CV-S
           UO-0:  0  -7.3               9.20749  CV-S
           UO-0:  0  -7.1               9.16770  CC-EPS-F
           UO-0:  0  -7.0               9.17275  CC-EPS-F
           UO-0:  0  -7.3               9.18463  CV-S
           UO-0:  0  -7.4               9.18423  CV-S
           UO-0:  0  -7.6               9.18555  CV-S
           UO-0:  0  -7.7               9.18516  CV-S
           UO-0:  0  -7.6               9.18502  CC-EPS-F
           UO-0:  0  -7.8               9.18708  CV-S
           UO-0:  0  -7.8               9.18347  CV-S
           UO-0:  0  -8.0               9.18746  CV-S
           UO-0:  0  -8.0               9.19233  CV-S
           UO-0:  0  -8.1               9.19418  CV-S
           UO-0:  1  -8.3               9.19412  CV-F-CC-DLT CG
           UO-0:  0  -8.4               9.19423  CV-

In [7]:
%%bash
. ~/.bashrc
cp /home/user/LOKT.02.048/week4/ACF.dat /home/user/LOKT.02.048/week4/ACF_PBE.dat
/home/user/LOKT.02.048/week4/bader -p all_atom -p atom_index NaCl-SIC.cube


  GRID BASED BADER ANALYSIS  (Version 0.28a 07/12/12)

  OPEN ... NaCl-SIC.cube       
  GAUSSIAN-STYLE INPUT FILE
  FFT-grid:   288 x 288 x 368
  CLOSE ... NaCl-SIC.cube       

  RUN TIME:   10.32 SECONDS

  CALCULATING BADER CHARGE DISTRIBUTION
                 0  10  25  50  75  100
  PERCENT DONE:  ********************** 

  REFINING AUTOMATICALLY
  ITERATION: 1
  EDGE POINTS:        437243
  REASSIGNED POINTS:   23895
  ITERATION: 2
  CHECKED POINTS:     240500
  REASSIGNED POINTS:       0

  RUN TIME:      85.01 SECONDS

  CALCULATING MINIMUM DISTANCES TO ATOMS
                 0  10  25  50  75  100
  PERCENT DONE:  **********************

  RUN TIME:   10.43 SECONDS

  WRITING BADER ATOMIC CHARGES TO ACF.dat
  WRITING BADER VOLUME CHARGES TO BCF.dat

  NUMBER OF BADER MAXIMA FOUND:              2
      SIGNIFICANT MAXIMA FOUND:              2
           NUMBER OF ELECTRONS:       27.99945

  WRITING ATOMIC VOLUMES 
                 0  10  25  50  75  100
  PERCENT DONE:  ****

bash: line 4: syntax error near unexpected token `('
bash: line 4: `f=open("ACF.dat").read()'


In [8]:
f=open("ACF.dat").read()
print(f)

    #         X           Y           Z        CHARGE     MIN DIST    ATOMIC VOL
 --------------------------------------------------------------------------------
    1      9.4486      9.4486      9.4486     10.0922      2.0720   3108.0624
    2      9.4486      9.4486     14.7777     17.9073      3.1912   5543.1327
 --------------------------------------------------------------------------------
    VACUUM CHARGE:               0.0000
    VACUUM VOLUME:               0.0000
    NUMBER OF ELECTRONS:        27.9995



In [9]:
calc = GPAW(xc='PBE0',
            mode='fd',
            h=0.14,
            nbands=16,
            eigensolver='rmm-diis',
            txt='NaCl-PBE0.txt',
            #mixer=Mixer(beta=0.05, nmaxold=5, weight=50.0),
            basis='dzp')
pair.set_calculator(calc)
pair.get_potential_energy()
pair.calc.write('NaCl-PBE0.gpw', mode='all')
density = calc.get_all_electron_density(gridrefinement=4) * Bohr**3
write('NaCl-PBE0.cube', pair, data=density)

/usr/lib/python3/dist-packages/gpaw/fd_operators.py:184: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  a_d, residual, rank, s = np.linalg.lstsq(A_md, [1, 1, 1, 0, 0, 0])


KeyboardInterrupt: 

In [ ]:
%%bash
. ~/.bashrc
cp /home/user/LOKT.02.048/week4/ACF.dat /home/user/LOKT.02.048/week4/ACF_SIC.dat
/home/user/LOKT.02.048/week4/bader -p all_atom -p atom_index NaCl-PBE0.cube

In [ ]:
f=open("ACF.dat").read()
print(f)